In [1]:
import laspy
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [17]:
# Step 1: Load .las Point Cloud using Laspy
def load_las(file_path):
    las = laspy.read(file_path)  # No need for `with` statement
    points = np.vstack((las.x, las.y, las.z)).transpose()
    return points

# Step 2: Convert to Open3D Point Cloud
def convert_to_open3d_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

# Step 3: Preprocess the Point Cloud
def preprocess_point_cloud(pcd, voxel_size=0.1):
    # Downsample
    downsampled_pcd = pcd.voxel_down_sample(voxel_size)
    # Estimate Normals
    downsampled_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=voxel_size * 2, max_nn=30))
    return downsampled_pcd

# Step 4: Perform Region Growing Segmentation
def region_growing_segmentation(pcd, eps=0.5, min_points=10):
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
    return labels

# Step 5: Assign Colors to Segments for Visualization
def color_segments(pcd, labels):
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0  # Unclustered points
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return max_label

# Step 6: Manual Labeling Function
def manual_labeling(pcd, labels):
    max_label = labels.max()
    cluster_labels = {}
    
    for cluster_idx in range(max_label + 1):
        # Extract each cluster
        cluster_points = pcd.select_by_index(np.where(labels == cluster_idx)[0])
        print(f"Displaying cluster {cluster_idx}. Assign a label (e.g., building, tree, etc.):")
        o3d.visualization.draw_geometries([cluster_points])
        
        # User input for labeling
        user_label = input(f"Enter label for cluster {cluster_idx}: ")
        cluster_labels[cluster_idx] = user_label

    return cluster_labels

def manual_labeling_with_context(full_pcd, labels):
    max_label = labels.max()
    cluster_labels = {}
    
    for cluster_idx in range(max_label + 1):
        # Extract cluster points
        cluster_points = full_pcd.select_by_index(np.where(labels == cluster_idx)[0])
        # Highlight the cluster in a different color while showing full context
        full_pcd_copy = full_pcd.select_by_index(np.arange(len(full_pcd.points)), invert=False)
        colors = np.asarray(full_pcd_copy.colors)
        colors[:, :] = [0.8, 0.8, 0.8]  # Set all points to gray for context
        colors[np.where(labels == cluster_idx)[0]] = [1, 0, 0]  # Highlight current cluster in red
        full_pcd_copy.colors = o3d.utility.Vector3dVector(colors)

        print(f"Displaying cluster {cluster_idx} with context. Assign a label (e.g., building, tree, etc.):")
        o3d.visualization.draw_geometries([full_pcd_copy])

        # User input for labeling
        user_label = input(f"Enter label for cluster {cluster_idx}: ")
        cluster_labels[cluster_idx] = user_label

    return cluster_labels

# Step 7: Save Labeled Data
def save_labeled_data(pcd, labels, cluster_labels, output_file="labeled_point_cloud.csv"):
    points = np.asarray(pcd.points)
    final_labels = [cluster_labels.get(label, "Unlabeled") for label in labels]
    
    # Save to CSV
    with open(output_file, "w") as f:
        f.write("x,y,z,label\n")
        for point, label in zip(points, final_labels):
            f.write(f"{point[0]},{point[1]},{point[2]},{label}\n")
    print(f"Labeled data saved to {output_file}")

In [6]:
data_folder = "../data/"
las_file = f"{data_folder}LAS/21/17/Run 172099_20231206_131419_0017_SPATIAL_SUBSAMPLED_2023-12-24_20h52_49_791.las"

In [36]:
# Load and preprocess the point cloud
points = load_las(las_file)
pcd = convert_to_open3d_point_cloud(points)
#pcd = preprocess_point_cloud(pcd, voxel_size=0.1)

In [39]:
# Perform region growing segmentation
labels = region_growing_segmentation(pcd, eps=0.5, min_points=100)

In [40]:
# Visualize clusters
max_label = color_segments(pcd, labels)
print(f"Visualizing {max_label + 1} clusters...")
o3d.visualization.draw_geometries([pcd])

Visualizing 610 clusters...
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [29]:
# Perform manual labeling
cluster_labels = manual_labeling_with_context(pcd, labels)

Displaying cluster 0 with context. Assign a label (e.g., building, tree, etc.):


KeyboardInterrupt: Interrupted by user

In [ ]:
# Save labeled data
save_labeled_data(pcd, labels, cluster_labels, output_file="labeled_point_cloud.csv")